<a href="https://colab.research.google.com/github/gayatri-bagusetty/Speech-Emotion-Recognition-Using-NLP-and-Hugging-Face-Transformers/blob/main/Speech_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pipwin

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8766 sha256=2de41c5acbe567d6e50aa0169c1989fa6cdba6ffab031308431389fe3398c3d5
  Stored in directory: /root/.cache/pip/wheels/07/45/2e/51ee464b7c3407327439df34c2fc62f978a1e5084aaf23d201
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=4560671bbb6b7d21d0e86818cd84f80578741ee207ffb1563dff670fc43de949
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25983 sha256=8162c5edac6dee75b4482525fe35bc23f389167bb373a2e2089e40528852343e
  Stored in directory: /root/.cache/pip/wheels/a5/9a/30/1003e89ab4555b81840ca46d361bf184f1e6ad880cae3b62a9
Successfully b

In [6]:
!apt-get install portaudio19-dev
!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (279 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [7]:
!pip install SpeechRecognition pyaudio

  Using cached SpeechRecognition-3.14.1-py3-none-any.whl.metadata (31 kB)
Using cached SpeechRecognition-3.14.1-py3-none-any.whl (32.9 MB)


In [3]:
!pip install --user -U nltk

**ONE AUDIO FILE AND TEXT**

In [10]:
import os
import string
import nltk
import torch
import speech_recognition as sr
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt_tab')

# Load Hugging Face emotion detection model
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

def preprocess_text(text):
    """Clean and preprocess text."""
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    words = word_tokenize(text)  # Tokenize words

    # Handle missing stopwords safely
    try:
        stop_words = set(stopwords.words('english')) - {"not", "no", "nor"}  # Keep negations
    except LookupError:
        nltk.download('stopwords')
        stop_words = set(stopwords.words('english')) - {"not", "no", "nor"}

    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)

def detect_emotion(text):
    """Detect emotion from text using Hugging Face model."""
    if not text:
        return "No text provided."

    # Preprocess text
    cleaned_text = preprocess_text(text)

    # Get emotion prediction
    emotion = emotion_classifier(cleaned_text)
    return emotion[0]['label']

def speech_to_text(audio_file):
    """Convert speech to text using SpeechRecognition."""
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(audio_file) as source:
            print("Listening...")
            audio = recognizer.record(source)

        text = recognizer.recognize_google(audio)
        print(f"Transcribed Text: {text}")
        return text
    except sr.UnknownValueError:
        return "Sorry, could not understand the audio."
    except sr.RequestError as e:
        return f"Speech Recognition API Error: {e}"

# Test Emotion Analysis
sample_texts = [
    "I am so happy today!",
    "I am really angry at you!",
    "That was a scary experience!",
    "I feel so loved and cherished.",
    "I am heartbroken and so sad."
]

for text in sample_texts:
    print(f"Input: {text} -> Emotion: {detect_emotion(text)}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cpu


Input: I am so happy today! -> Emotion: joy
Input: I am really angry at you! -> Emotion: anger
Input: That was a scary experience! -> Emotion: fear
Input: I feel so loved and cherished. -> Emotion: love
Input: I am heartbroken and so sad. -> Emotion: sadness


**DATASET**

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
import string
import nltk
import torch
import speech_recognition as sr
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline
from datetime import datetime  # Timestamp handling

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Load Hugging Face emotion detection model
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

def preprocess_text(text):
    """Clean and preprocess text."""
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    words = word_tokenize(text)  # Tokenize words

    # Handle missing stopwords safely
    try:
        stop_words = set(stopwords.words('english')) - {"not", "no", "nor"}  # Keep negations
    except LookupError:
        nltk.download('stopwords')
        stop_words = set(stopwords.words('english')) - {"not", "no", "nor"}

    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)

def detect_emotion(text):
    """Detect emotion from text using Hugging Face model."""
    if not text:
        return "No text provided."

    # Preprocess text
    cleaned_text = preprocess_text(text)

    # Get emotion prediction
    emotion = emotion_classifier(cleaned_text)
    return emotion[0]['label']

def speech_to_text(audio_file):
    """Convert speech to text from an audio file."""
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_file) as source:
            print(f"🎤 Processing: {audio_file}")
            audio = recognizer.record(source)

        text = recognizer.recognize_google(audio)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Get current timestamp
        return text, timestamp
    except sr.UnknownValueError:
        return "Could not understand the audio.", None
    except sr.RequestError as e:
        return f"Speech Recognition API Error: {e}", None

def process_audio_dataset(dataset_folder):
    """Process multiple audio files in a dataset folder."""
    if not os.path.exists(dataset_folder):
        print("Dataset folder not found.")
        return

    for file in os.listdir(dataset_folder):
        if file.endswith(".wav") or file.endswith(".mp3"):
            audio_path = os.path.join(dataset_folder, file)
            spoken_text, speech_time = speech_to_text(audio_path)
            if spoken_text and speech_time:
                detected_emotion = detect_emotion(spoken_text)
                print(f"📝 File: {file} | ⏱️ Timestamp: {speech_time} | 🗣️ Text: {spoken_text} | 💡 Emotion: {detected_emotion}")

# Set the path to your dataset folder
dataset_folder = "/content/drive/MyDrive/NLP"  # Change this to your actual folder path
process_audio_dataset(dataset_folder)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu


🎤 Processing: /content/drive/MyDrive/NLP/OSR_us_000_0011_8k.wav
📝 File: OSR_us_000_0011_8k.wav | ⏱️ Timestamp: 2025-02-16 14:35:20 | 🗣️ Text: the boy was there when the sun rose a rod is used to catch pink salmon the source of the huge river is the Clear Spring kick the ball straight and follow through help the women get back to her feet a pot of tea helps to pass the evening Smoky fires black flame in heat the soft cushion broke the man's fall the salt Breeze came across the sea the girl at the booth sold 50 bonds | 💡 Emotion: sadness
🎤 Processing: /content/drive/MyDrive/NLP/OSR_us_000_0061_8k.wav
📝 File: OSR_us_000_0061_8k.wav | ⏱️ Timestamp: 2025-02-16 14:35:38 | 🗣️ Text: the mute muffled the high tones of the horn the gold ring fits only appears to ear the old pan was covered with hard fudge watch the log float in the wide river the node on the stock of wheat grow daily the Heap of fallen leaves was set on fire right fast if you want to finish early his shirt was clean but one butt